# Short intro to Keras

Keras is a great Python library and a wrapper for existing neural network packages. It runs with either the `theano` or `tensorflow` backend libraries.

Keras implements two main approaches:
    
    * the Sequential model
    * the functional API

The main type of model is the **Sequential** model, a linear stack of layers. You will only need the functional API for more complex models.

### The Sequential model

In [2]:
from keras.models import Sequential

model = Sequential()

Using TensorFlow backend.


Sequential means that it takes a list of layers. You can add one layer at a time to the model. Here is a simple model:

In [3]:
from keras.layers.core import Dense, Activation

model.add(Dense(input_dim=100, units=64))
model.add(Activation("relu"))
model.add(Dense(units=10))
model.add(Activation("softmax"))


Instructions for updating:
Colocations handled automatically by placer.


For the first layer you need to specify its dimensions, the remaining layers will infer the size.

Alternative, equivalent formulation:

In [4]:
model = Sequential()
model.add(Dense(input_dim=100, units=64))
model.add(Activation("relu"))
model.add(Dense(units=10, activation='softmax'))

Then you can compile and train the model.

In [5]:
model.compile(loss='categorical_crossentropy', optimizer="sgd", metrics=['accuracy'])

A complicated LSTM model is easly created: 

In [6]:
from keras.layers import Embedding, LSTM, Dropout

num_labels=2
vocabulary_size=10000
model = Sequential()
model.add(Embedding(output_dim=128, input_dim=vocabulary_size, input_length=100))
model.add(LSTM(units=64, activation='tanh'))
model.add(Dropout(rate=0.2))
model.add(Dense(num_labels))
model.add(Activation('softmax'))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


### Functional API

In the functional API each layer is a function and can be applied to another layer.

In [7]:
from keras.models import Model
from keras.layers import Input, Flatten

# input: a sequence  of 5 integers, each representing a word (index between 0 and vocab_size).
main_input = Input(shape=(5,), dtype='int32', name='main_input')

# now the embedding layer will encode the input sequence
# into a sequence of dense 128-dimensional vectors.
embeds = Embedding(output_dim=128, input_dim=vocabulary_size, input_length=5)(main_input)
flatten = Flatten()(embeds) # we flatten it as Dense expects a 2D input
dense = Dense(64, activation='tanh')(flatten)

# finally the softmax (logistic) output layer
main_loss = Dense(num_labels, activation='softmax', name='main_output')(dense)

# the model is specified by connecting input and output
model = Model(inputs=[main_input], outputs=[main_loss])

With the functional API you can create a lot fun models quickly. 

# A Multilayer-Perceptron/Feedforward Neural Network

In [11]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
import keras

max_features = 2000

# read in the training data
train_data = pd.read_csv('../data/sa_train.csv')
print(len(train_data), train_data['output'].unique())

# create TFIDF representations
vectorizer = TfidfVectorizer(ngram_range=(1,2), min_df=0.001, max_df=0.75, stop_words='english', max_features=max_features)

X_train = vectorizer.fit_transform(train_data.input)
print(X_train.shape)

# transform labels into numbers
labels2numbers = LabelEncoder()

y_train_org = labels2numbers.fit_transform(train_data['output'])
print(y_train_org[:10], len(y_train_org))



# read in test data
test_data = pd.read_csv('../../lectures/17/sa_test.csv')
print(len(test_data), test_data['output'].unique())

X_test = vectorizer.transform(test_data.input)
print(X_test.shape)

y_test_org = labels2numbers.transform(test_data['output'])
print(y_test_org[:10], len(y_test_org))

# get number of classes for transformation
num_classes = max(y_train_org) + 1

print('Convert class vector to binary 1-hot encoding matrix (for use with categorical_crossentropy)')
y_train = keras.utils.to_categorical(y_train_org, num_classes)
y_test = keras.utils.to_categorical(y_test_org, num_classes)

1800 ['neg' 'pos']
(1800, 2000)
[0 0 0 1 1 0 1 1 0 0] 1800
200 ['pos' 'neg']
(200, 2000)
[1 0 0 0 1 0 0 1 0 0] 200
Convert class vector to binary 1-hot encoding matrix (for use with categorical_crossentropy)


In [12]:
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

print(y_train[:10], y_train_org[:10])

y_train shape: (1800, 2)
y_test shape: (200, 2)
[[1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]] [0 0 0 1 1 0 1 1 0 0]


In [13]:
batch_size = 8
epochs = 5

print('Building model...')
model = Sequential()
model.add(Dense(512, input_shape=(max_features,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)
loss, accuracy = model.evaluate(X_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

Building model...
Instructions for updating:
Use tf.cast instead.
Train on 1620 samples, validate on 180 samples
Epoch 1/5
1620/1620 [==============================] - 6s 3ms/step - loss: 0.5610 - acc: 0.7185 - val_loss: 0.3878 - val_acc: 0.8556
Epoch 2/5
1620/1620 [==============================] - 5s 3ms/step - loss: 0.2380 - acc: 0.9216 - val_loss: 0.3258 - val_acc: 0.8611
Epoch 3/5
1620/1620 [==============================] - 5s 3ms/step - loss: 0.1108 - acc: 0.9704 - val_loss: 0.3819 - val_acc: 0.8389
Epoch 4/5
1620/1620 [==============================] - 5s 3ms/step - loss: 0.0504 - acc: 0.9920 - val_loss: 0.3876 - val_acc: 0.8556
Epoch 5/5
200/200 [==============================] - 0s 293us/step
Test loss: 0.5518322148919106
Test accuracy: 0.78


In [14]:
# output class probability matrix
print(model.predict(X_test[:10]))

[[6.6256290e-04 9.9933738e-01]
 [9.9985409e-01 1.4590834e-04]
 [8.7059714e-02 9.1294026e-01]
 [9.9940169e-01 5.9825863e-04]
 [1.5189968e-03 9.9848098e-01]
 [9.3822020e-01 6.1779819e-02]
 [5.7638186e-01 4.2361814e-01]
 [8.9618921e-01 1.0381078e-01]
 [9.9939859e-01 6.0137175e-04]
 [4.3675789e-01 5.6324208e-01]]


In [15]:
# output actual class predictions
model.predict_classes(X_test[:10])

array([1, 0, 1, 0, 1, 0, 0, 0, 0, 1])

In [16]:
from sklearn.metrics import f1_score, classification_report
print(classification_report(y_test_org, model.predict_classes(X_test)))

              precision    recall  f1-score   support

           0       0.82      0.77      0.80       111
           1       0.74      0.79      0.76        89

   micro avg       0.78      0.78      0.78       200
   macro avg       0.78      0.78      0.78       200
weighted avg       0.78      0.78      0.78       200



## References

* [http://keras.io/#getting-started-30-seconds-to-keras](http://keras.io/#getting-started-30-seconds-to-keras)
* [http://keras.io/getting-started/sequential-model-guide/](http://keras.io/getting-started/sequential-model-guide/)
* [https://arxiv.org/pdf/1510.00726.pdf](https://arxiv.org/pdf/1510.00726.pdf)